In [1]:
import sys
print(sys.executable)

c:\Projects\DeepFake_Tumor\Detection-of-Tumor-Manipulation-in-MRI-Scans\venv\Scripts\python.exe


In [3]:
import torch
print(torch.cuda.is_available())

True


In [4]:
import os
print(os.listdir("Dataset"))

['Processed', 'Raw']


In [11]:
import os
import cv2
import random
import numpy as np
from tqdm import tqdm
BASE_DIR = os.getcwd()

RAW_PATH = os.path.join(BASE_DIR, "Dataset", "Raw")
PROCESSED_PATH = os.path.join(BASE_DIR, "Dataset", "Processed")

print("RAW PATH:", RAW_PATH)
print("PROCESSED PATH:", PROCESSED_PATH)

RAW PATH: c:\Projects\DeepFake_Tumor\Detection-of-Tumor-Manipulation-in-MRI-Scans\Dataset\Raw
PROCESSED PATH: c:\Projects\DeepFake_Tumor\Detection-of-Tumor-Manipulation-in-MRI-Scans\Dataset\Processed


In [12]:
print(os.listdir(RAW_PATH))

['Testing', 'Training']


In [13]:
def extract_random_patch(img):
    h, w = img.shape[:2]

    center_x = w // 2
    center_y = h // 2
    radius = random.randint(20, 40)

    mask = np.zeros((h, w), dtype=np.uint8)
    cv2.circle(mask, (center_x, center_y), radius, 255, -1)

    patch = cv2.bitwise_and(img, img, mask=mask)

    return patch, mask

In [14]:
def insert_patch(target_img, patch, mask):
    h, w = target_img.shape[:2]

    x_offset = random.randint(30, w - 30)
    y_offset = random.randint(30, h - 30)

    # Smooth mask edges
    smooth_mask = cv2.GaussianBlur(mask, (31, 31), 0)

    for c in range(3):
        target_img[:, :, c] = np.where(
            smooth_mask > 0,
            patch[:, :, c],
            target_img[:, :, c]
        )

    return target_img

In [15]:
def generate_deepfake(split_type):
    print(f"\nGenerating deepfake for {split_type} set...")

    raw_split_path = os.path.join(RAW_PATH, split_type)
    processed_split_path = os.path.join(PROCESSED_PATH, split_type.lower())

    real_output = os.path.join(processed_split_path, "real")
    fake_output = os.path.join(processed_split_path, "manipulated")

    os.makedirs(real_output, exist_ok=True)
    os.makedirs(fake_output, exist_ok=True)

    tumor_images = []
    for cls in tumor_classes:
        class_path = os.path.join(raw_split_path, cls)
        for file in os.listdir(class_path):
            tumor_images.append(os.path.join(class_path, file))

    non_tumor_path = os.path.join(raw_split_path, non_tumor_class)
    non_tumor_images = os.listdir(non_tumor_path)

    for i in tqdm(range(len(non_tumor_images))):
        non_tumor_img_path = os.path.join(non_tumor_path, non_tumor_images[i])
        non_tumor_img = cv2.imread(non_tumor_img_path)

        if non_tumor_img is None:
            continue

        tumor_img_path = random.choice(tumor_images)
        tumor_img = cv2.imread(tumor_img_path)

        if tumor_img is None:
            continue

        non_tumor_img = cv2.resize(non_tumor_img, (224, 224))
        tumor_img = cv2.resize(tumor_img, (224, 224))

        patch, mask = extract_random_patch(tumor_img)
        deepfake_img = insert_patch(non_tumor_img.copy(), patch, mask)

        cv2.imwrite(os.path.join(real_output, f"real_{i}.jpg"), non_tumor_img)
        cv2.imwrite(os.path.join(fake_output, f"deepfake_{i}.jpg"), deepfake_img)

    print(f"Deepfake generation completed for {split_type}.")

In [16]:
import os
print(os.getcwd())

c:\Projects\DeepFake_Tumor\Detection-of-Tumor-Manipulation-in-MRI-Scans


In [17]:
generate_deepfake("Training")



Generating deepfake for Training set...


100%|██████████| 1400/1400 [00:35<00:00, 39.91it/s]

Deepfake generation completed for Training.


In [18]:
generate_deepfake("Testing")


Generating deepfake for Testing set...


100%|██████████| 400/400 [00:09<00:00, 41.93it/s]

Deepfake generation completed for Testing.


In [19]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

train_dir = "Dataset/Processed/train"
test_dir = "Dataset/Processed/test"

train_dataset = datasets.ImageFolder(train_dir, transform=transform)
test_dataset = datasets.ImageFolder(test_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=0)

print("Classes:", train_dataset.classes)

Classes: ['manipulated', 'real']


In [20]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(weights="IMAGENET1K_V1")

for param in model.parameters():
    param.requires_grad = False

num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.0001)

print("Model Ready on:", device)

Model Ready on: cuda


In [ ]:
from tqdm import tqdm

num_epochs = 15

for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    print(f"\nEpoch [{epoch+1}/{num_epochs}]")

    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print("Loss:", running_loss / len(train_loader))
    print("Train Accuracy:", 100 * correct / total)


Epoch [1/10]


100%|██████████| 1400/1400 [02:46<00:00,  8.39it/s]


Loss: 0.7106098677856582
Train Accuracy: 50.99107142857143

Epoch [2/10]


100%|██████████| 1400/1400 [01:19<00:00, 17.56it/s]


Loss: 0.6950577895343304
Train Accuracy: 52.357142857142854

Epoch [3/10]


...

Loss: 0.6788155511447361
Train Accuracy: 55.276785714285715

Epoch [9/10]


100%|██████████| 1400/1400 [01:12<00:00, 19.21it/s]


Loss: 0.6774646435252257
Train Accuracy: 55.267857142857146

Epoch [10/10]


100%|██████████| 1400/1400 [01:11<00:00, 19.51it/s]

Loss: 0.6767677692643234
Train Accuracy: 55.482142857142854
